In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset 

In [4]:
from mlpack.utils import to_device, to_fp16

# Data

In [ ]:
train_id = pd.read_csv('../datasets/ENEL/log_norm_clean_data_train.csv', usecols=[0], sep=',')
valid_id = pd.read_csv('../datasets/ENEL/log_norm_clean_data_valid.csv', usecols=[0], sep=',')

In [ ]:
df = pd.read_csv('../datasets/ENEL/dataset.csv', sep='\t', index_col=0)
df.head()

In [ ]:
df_train = df.loc[train_id.values[:,0]]

In [ ]:
df_valid = df.loc[valid_id.values[:,0]]

In [ ]:
x_train = df_train.values[:,:-1]
y_train = df_train.values[:,-1]

x_valid = df_valid.values[:,:-1]
y_valid = df_valid.values[:,-1]

# Dataset

In [ ]:
class ENELDataset(Dataset):
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx].astype(np.float32), self.y[idx].astype(np.int64)

In [ ]:
ds_train = ENELDataset(x_train, y_train)
ds_valid = ENELDataset(x_valid, y_valid)

In [ ]:
dl_train = DataLoader(ds_train, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
dl_valid = DataLoader(ds_valid, batch_size=32, shuffle=False, pin_memory=True, num_workers=4)

# LSTM

In [13]:
class MyLSTM(nn.Module):
    
    lstm_kwargs = {
        'input_size': 1,
        'hidden_size': 128,
        'num_layers': 2,
        'batch_first': True,
        'dropout': 0.1,
        'bidirectional': True
    }
    
    def __init__(self, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(**self.lstm_kwargs)
        
        self.linear_size = self.lstm_kwargs['hidden_size'] * self.lstm_kwargs['num_layers']
        if self.lstm_kwargs['bidirectional']:
            self.linear_size *= 2
        
        self.linear = nn.Linear(self.linear_size, num_classes)
        
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h = h.permute(1, 0, 2).reshape(-1, self.linear_size)
        o = self.linear(h)
        return o
        

In [14]:
class LSTMSelfAttetion(nn.Module):
    
    lstm_kwargs = {
        'input_size': 1,
        'hidden_size': 128,
        'num_layers': 2,
        'batch_first': True,
        'dropout': 0.1,
        'bidirectional': True
    }
    
    def __init__(self, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(**self.lstm_kwargs)
        
        att_sz = self.lstm_kwargs['hidden_size']
        if self.lstm_kwargs['bidirectional']:
            att_sz *= 2
        
        self.query = nn.Linear(att_sz, 1)
        self.key = nn.Linear(att_sz, 1)
        self.value = nn.Linear(att_sz, 1)
        
        self.classifier = nn.Linear(1034, 2)
        
    def forward(self, x):
        o, _ = self.lstm(x)
        q = self.query(o)
        k = self.key(o)
        v = self.value(o)
        z = torch.matmul(q, k.permute(0, 2, 1)) @ v
        z = z.squeeze()
        o = self.classifier(z)
        
        return o


In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Eval

In [16]:
def dl_generator(dataloader):
    for x, y in dataloader:
        x = x.unsqueeze(-1)
        x, y = to_device(x, y, device=device)
        yield x, y

In [22]:
def evaluate_fn(model, dataloader, loss_fn):
    model.eval()
    losses = []
    preds = []
    trues = []
    for inputs in tqdm(dataloader, leave=False, desc='Eval...', total=len(dataloader)):
        o = model(**inputs)
        loss = loss_fn(o, y)
        
        preds += o.argmax(1).detach().cpu().numpy().tolist()
        trues += y.detach().cpu().numpy().tolist()
        losses.append(loss.item())
        
    acc = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds)
    conf = confusion_matrix(trues, preds)
    
    print('--- Validation ---')
    print(f'F1 = {f1}\t Acc = {acc}')
    print(conf)
    return np.array(losses).mean(), f1       

In [18]:
model = LSTMSelfAttetion()

In [19]:
model.to(device)

LSTMSelfAttetion(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (query): Linear(in_features=256, out_features=1, bias=True)
  (key): Linear(in_features=256, out_features=1, bias=True)
  (value): Linear(in_features=256, out_features=1, bias=True)
  (classifier): Linear(in_features=1034, out_features=2, bias=True)
)

# Optim

In [20]:
loss_fn = nn.CrossEntropyLoss(torch.tensor([.5, 1.]).to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

In [21]:
evaluate_fn(model, dl_valid, loss_fn)

--- Validation ---
F1 = 0.15138482711164628	 Acc = 0.08189093616229294
<function confusion_matrix at 0x7f8773f66e60>


(tensor(0.6981, device='cuda:0', grad_fn=<NllLossBackward>),
 0.15138482711164628)